In [6]:
# Imports

import os
import numpy as np
import polars as pl
from typing import Tuple
from geopy.distance import geodesic

In [7]:
# Paths

CSV_EXTENSION = '.csv'

POSITION_DATA_2021 = '2021-01-01_2021-12-31' + CSV_EXTENSION
POSITION_DATA_2022 = '2022-01-01_2022-12-31' + CSV_EXTENSION
POSITION_DATA_2023 = '2023-01-01_2023-12-31' + CSV_EXTENSION
POSITION_DATA_2024 = '2024-01-01_2024-07-09' + CSV_EXTENSION

VEHICLE_INFORMATION_2021 = 'Kjøretøysdata 010121 - 311221' + CSV_EXTENSION
VEHICLE_INFORMATION_2022 = 'Kjøretøysdata 010122 - 311222' + CSV_EXTENSION
VEHICLE_INFORMATION_2023 = 'Kjøretøysdata 010123 - 311223' + CSV_EXTENSION
VEHICLE_INFORMATION_2024 = 'Kjøretøysdata 010124 - 220624' + CSV_EXTENSION

current_dir = os.getcwd()

years = (2021, 2022, 2023, 2024)

relative_path_position_data = os.path.join('..', 'LINX-data', 'posisjonsdata')
relative_path_vehicle_information = os.path.join('..', 'LINX-data', 'kjøretøysdata')
relative_path_results = os.path.join('..', 'resultater')
relative_path_ekvipasjer = os.path.join(relative_path_results, 'ekvipasjer' + CSV_EXTENSION)

path_ekvipasjer = os.path.join(current_dir, relative_path_ekvipasjer)

path_position_data_2021 = os.path.join(current_dir, relative_path_position_data, POSITION_DATA_2021)
path_position_data_2022 = os.path.join(current_dir, relative_path_position_data, POSITION_DATA_2022)
path_position_data_2023 = os.path.join(current_dir, relative_path_position_data, POSITION_DATA_2023)
path_position_data_2024 = os.path.join(current_dir, relative_path_position_data, POSITION_DATA_2024)

path_vehicle_information_2021 = os.path.join(current_dir, relative_path_vehicle_information, VEHICLE_INFORMATION_2021)
path_vehicle_information_2022 = os.path.join(current_dir, relative_path_vehicle_information, VEHICLE_INFORMATION_2022)
path_vehicle_information_2023 = os.path.join(current_dir, relative_path_vehicle_information, VEHICLE_INFORMATION_2023)
path_vehicle_information_2024 = os.path.join(current_dir, relative_path_vehicle_information, VEHICLE_INFORMATION_2024)

path_ekvipasjer_km_vekt_turer = os.path.join(current_dir, relative_path_results, 'ekvipasjer_km_vekt_turer' + CSV_EXTENSION)

paths_position_information = (path_position_data_2021, path_position_data_2022, path_position_data_2023, path_position_data_2024)
paths_vehicle_information = (path_vehicle_information_2021, path_vehicle_information_2022, path_vehicle_information_2023, path_vehicle_information_2024)

In [8]:
# Ekvipasjer

ekvipasjer = (
    '3-akslet trekkvogn med 4-akslet tilhenger',
    '3-akslet trekkvogn med 5-akslet tilhenger',
    '4-akslet trekkvogn med 4-akslet tilhenger',
    '4-akslet trekkvogn med 5-akslet tilhenger'
)

In [9]:
def distance_weight_rides(
        ekvipasje: str, 
        year: int, 
        df_ekvipasjer: pl.DataFrame,
        df_position_information: pl.DataFrame, 
        df_vechicle_information: pl.DataFrame) -> Tuple[float, float, float]:
    '''
    Finner total tilbakelagt distanse, gjennomsnittsvekt, og antall turer for en spesifisert ekvipasje innenfor et spesifisert år. 
    '''

    def VINS_ekvipasje_year(ekvipasje: str, year: int, df_ekvipasjer: pl.DataFrame):
        '''
        Returnerer VIN nummer for lastebiler og tilhengere innen spesifisert ekvipasje som har blitt brukt i det spesifiserte året.
        '''

        df_year = df_ekvipasjer.filter((pl.col('år') == year) & (pl.col('ekvipasje') == ekvipasje))
        vins_trucks = df_year.select(pl.col('VIN_lastebil')).to_series().to_list()
        vins_tilhengere = df_year.select(pl.col('VIN_tilhenger')).to_series().to_list()

        return vins_trucks, vins_tilhengere

    def get_distance(vins: Tuple[str, ...], df_position_information: pl.DataFrame):
        '''
        Returnerer totalt tilbakelagt distanse for kjøretøy med spesifiserte VIN-numre for det året posisjonsrapporten gjelder. 

        Parameters
        ----------
        vins
            VIN-numre tilhørende kjøretøyene man ønsker å finne sammenlagt distanse for.
        df_position_information
            Posisjonsrapport som dataframe. Posisjonsrapportene gjelder for enkeltår.  
        '''
        
        total_distance = 0

        for vin in vins:
            
            df_position_vin = df_position_information.filter(pl.col('VIN') == vin)
            df_position_vin_sorted_date = df_position_vin.sort(by='Dato')

            total_distance_vin = 0

            for i in range(1, df_position_vin_sorted_date.height):
                start_point = (df_position_vin_sorted_date[i - 1, 'Latitude'], df_position_vin_sorted_date[i - 1, 'Longitude'])
                end_point = (df_position_vin_sorted_date[i, 'Latitude'], df_position_vin_sorted_date[i, 'Longitude'])
                distance = geodesic(start_point, end_point).kilometers
                total_distance_vin += distance

            total_distance += total_distance_vin

        return total_distance

    def get_weight(vins: Tuple[str, ...], df_vechicle_information: pl.DataFrame):
        '''
        Returnerer gjennomsnittlig max-vekt for kjøretøy med spesifiserte VIN-nummer.
        Gjelder for enkeltår, ettersom kjøretøysrapporten gjelder for enkeltår.
        '''

        df_vehicle_vin = df_vechicle_information.filter(pl.col('VIN').is_in(vins))
        df_vehicle_vin = df_vehicle_vin.with_columns(pl.col('Max vekt').cast(pl.Float64))
        df_vehicle_vin = df_vehicle_vin.filter((pl.col('Max vekt').is_not_nan()) & (pl.col('Max vekt').is_not_null()))

        max_weights = df_vehicle_vin.select(pl.col('Max vekt')).to_numpy()

        average_max_weight = np.sum(max_weights) / len(max_weights)

        return average_max_weight

    def get_rides(vins: Tuple[str, ...], df_vechicle_information: pl.DataFrame):
        '''
        Returnerer antall turer kjørt av kjøretøy med spesifiserte VIN-nummer.
        Gjelder for enkeltår ettersom kjøretøysdataen gjelder for enkeltår. 
        '''        

        df_vehicle_vin = df_vechicle_information.filter(pl.col('VIN').is_in(vins))
        df_vehicle_vin = df_vehicle_vin.with_columns(pl.col('Distanse (km)').cast(pl.Float64))
        df_vehicle_vin_valid_rides = df_vehicle_vin.filter((pl.col('Distanse (km)').is_not_nan()) & (pl.col('Distanse (km)').is_not_null()) & (pl.col('Distanse (km)') > 0))

        number_of_rides = len(df_vehicle_vin_valid_rides)

        return number_of_rides

    vins_trucks, vins_trailers = VINS_ekvipasje_year(ekvipasje, year, df_ekvipasjer)

    distance = get_distance(vins_trailers, df_position_information)
    weight = get_weight(vins_trucks, df_vechicle_information)
    rides = get_rides(vins_trucks, df_vechicle_information)

    return distance, weight, rides


In [10]:
schema = (
    'Ekvipasje',
    'år',
    'distanse',
    'vekt',
    'turer'
)

data = []

# position_information og vehicle_information gjelder begge for enkeltår
for year, path_position_information, path_vehicle_information in zip(years, paths_position_information, paths_vehicle_information):

    df_ekvipasjer = pl.read_csv(path_ekvipasjer, truncate_ragged_lines=True, ignore_errors=True)
    df_position_information = pl.read_csv(path_position_information, truncate_ragged_lines=True, ignore_errors=True)
    df_vehicle_information = pl.read_csv(path_vehicle_information, truncate_ragged_lines=True, ignore_errors=True, separator=';', decimal_comma=True)

    for ekvipasje in ekvipasjer:
        
        distance, weight, rides = distance_weight_rides(ekvipasje, year, df_ekvipasjer, df_position_information, df_vehicle_information)

        data.append([ekvipasje, year, distance, weight, rides])

df_results = pl.DataFrame(schema=schema, data=data)

df_results.write_csv(path_ekvipasjer_km_vekt_turer)

C:\Users\ander\AppData\Local\Temp\ipykernel_15168\3241922037.py:65: RuntimeWarning: invalid value encountered in scalar divide
  average_max_weight = np.sum(max_weights) / len(max_weights)
